In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
import lightgbm

In [ ]:
all_csv = pd.read_csv('cleaned.csv.gz',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')

In [ ]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [ ]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

filter_by_stores = ['1', '2']  # note: please use string here (unlike Mine.ipynb)
filter_by_family = ['DAIRY', 'PRODUCE']
#filter_by_family = ['']
#filter_by_dates = '2014-06-05'

In [ ]:
if filter_by_dates == None:
    train_start_date = '2013-01-01'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [ ]:
if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [ ]:
all.info(verbose=True)

## One Hot Encoding

In [ ]:
def one_hot_encode(df):
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday'])    

In [ ]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

## Experiment I -- Linear Regression

In [ ]:
run_experiment_I = True
if run_experiment_I:
    lr = LinearRegression()
    lr.fit(X_train, y_train)

In [ ]:
if run_experiment_I:
    y_pred_train = lr.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = lr.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

## Experiment II -- Lightgbm

In [ ]:
lgb_params = {
    'boosting_type' : 'gbdt',  # gradient boosting decision tree
    'early_stopping_rounds': 200,
    'force_col_wise': True,
    'learning_rate': 0.1,
    'max_depth': 10,
    'metric': 'mse',  # mean square error
    'num_iterations': 5000,
    'num_leaves': 10,
    'random_state': 1,
    'verbose': 0
}

run_experiment_II = True  # should be true
if run_experiment_II:
    X_train_lgb = lightgbm.Dataset(data=X_train, label=y_train, feature_name='auto')
    X_val_lgb = lightgbm.Dataset(data=X_val, label=y_val, reference=X_train_lgb, feature_name='auto')

In [ ]:
if run_experiment_II:
    lgb = lightgbm.train(
        params=lgb_params, 
        train_set=X_train_lgb,
        valid_sets=[X_train_lgb, X_val_lgb],
    )

In [ ]:
if run_experiment_II:
    y_pred_train = lgb.predict(X_train, num_iteration=lgb.best_iteration)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = lgb.predict(X_val, num_iteration=lgb.best_iteration)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

## Experiment III -- Random Forest (Too Slow)

In [ ]:
run_experiment_III = True
if run_experiment_III:
    random_forest = RandomForestRegressor(random_state=1)
    random_forest.fit(X_train, y_train.values.ravel())

In [ ]:
if run_experiment_III:
    y_pred_train = random_forest.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = random_forest.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

## Experiment IV -- XGBoost

In [ ]:
run_experiment_IV = True
if run_experiment_IV:
    xgb = xgboost.XGBRegressor(n_estimators=20, early_stopping_rounds=100, learning_rate=0.3)
    xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True)

In [ ]:
if run_experiment_IV:
    y_pred_train = xgb.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = xgb.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

In [ ]:
if run_experiment_IV:
    xgb.save_model("xgb.model")

## Experiment V -- SVR

In [ ]:
from sklearn.svm import SVR
run_experiment_V = False  # slow
if run_experiment_V:
    # svr = SVR(C=1.0, epsilon=0.2)
    svr = SVR(kernel='rbf')
    svr.fit(X_train, y_train.values.ravel())

In [ ]:
if run_experiment_V:
    y_pred_train = svr.predict(X_train)
    y_pred_train[y_pred_train < 0] = 0
    y_pred_val = svr.predict(X_val)
    y_pred_val[y_pred_val < 0] = 0

    print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
    print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))
    print("RMS log-error train (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print("RMS log-error val (actual): ",
          np.sqrt(mean_squared_log_error(np.expm1(y_val), np.expm1(y_pred_val))))

## Test (Moment of Truth)

In [ ]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [ ]:
y_pred_test = main_predict(lgb, X_test)

In [ ]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
#submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
#submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': max(y_pred_test, 0)})
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submission.csv', index=False)